In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
csv_file = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(csv_file)
raw = pd.read_csv(csv_file)

In [7]:
# Head for reference
raw.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


# Player Count

In [3]:
# Counting the number of unique SN assumed to be ID names
player_count = pd.DataFrame(data = {'Total Number of Players' : raw['SN'].nunique()}, index = [0])
#player_count.head()

# Purchasing Analysis

In [4]:
# Counting the unique number of item IDs
purchase_analysis = pd.DataFrame(data = {'Number of Unique Items' : raw['Item ID'].nunique()}, index = [0])

# Taking the mean of the item purchase price
purchase_analysis['Average Purchase Price'] = raw['Price'].mean()

#Changing the format to be currency
purchase_analysis['Average Purchase Price'] = purchase_analysis['Average Purchase Price'].map('${:.2f}'.format)

# Calculating the number of unique purchases
purchase_analysis['Total Number of Purchases'] = raw['Purchase ID'].nunique()

#Calculating the sum of all of the purchases
purchase_analysis['Total Revenue'] = raw['Price'].sum()

# Changing the price to be 
purchase_analysis['Total Revenue'] = purchase_analysis['Total Revenue'].map('${:,.2f}'.format)
#purchase_analysis.head()

# Gender Demographics

# Purchasing Analysis (Gender)

In [5]:
# Generating a data frame with the number of players grouped by gender
grouped_gender = raw.groupby('Gender')
gender_demographics = pd.DataFrame(grouped_gender.nunique()['SN'])
gender_demographics = gender_demographics.rename(columns = {'SN' : 'Total Count'})

# Calculating the percentage of players by gender
gender_demographics['Percentage of Players'] = gender_demographics['Total Count'].div(gender_demographics['Total Count'].sum())*100
gender_demographics['Percentage of Players'] = gender_demographics['Percentage of Players'].map('{:.2f}%'.format)

# Sorting the number of players in descending order
gender_demographics = gender_demographics.sort_values(["Total Count"], ascending = False)
#gender_demographics.head()

In [18]:
# Generating a data frame with the number of purchases grouped by gender
purchasing_analysis = pd.DataFrame(grouped_gender.count()['Item ID'])
purchasing_analysis = purchasing_analysis.rename(columns = {'Item ID' : 'Purchase Count'})

# Calculating the average purchase price by gender
purchasing_analysis['Average Purchase Price'] = grouped_gender.mean()['Price']
purchasing_analysis['Average Purchase Price'] = purchasing_analysis['Average Purchase Price'].map('${:,.2f}'.format)

# Calculating the Average value of the purchases by gender
purchasing_analysis['Average Purchase Value'] = grouped_gender.sum()['Price']
purchasing_analysis['Average Purchase Value'] = purchasing_analysis['Average Purchase Value'].map('${:,.2f}'.format) 

# Calculating the average total purchase per person by gender
#purchasing_analysis_raw = raw.loc[:, ['Gender', 'Price']]
#purchasing_analysis_raw = purchasing_analysis_raw.groupby('Gender').sum()
#purchasing_analysis = purchasing_analysis.join(purchasing_analysis_raw)
#purchasing_analysis = purchasing_analysis.rename(columns = {'Price' : 'Total Purchase Value'})
#purchasing_analysis['Total Purchase Value'] = purchasing_analysis['Total Purchase Value'].map('${:,.2f}'.format) 
purchasing_analysis.head()

,Purchase Count,Average Purchase Price,Average Purchase Value
Gender,,,
Female,113,$3.20,$361.94
Male,652,$3.02,"$1,967.64"
Other / Non-Disclosed,15,$3.35,$50.19
